# Setup

In [2]:
# Check out the robocon2017 branch of RigidBodyDynamics and RigidBodyTreeInspector:
Pkg.checkout("RigidBodyDynamics", "robocon2017")
Pkg.checkout("RigidBodyTreeInspector", "robocon2017")

using RigidBodyDynamics
using StaticArrays
using Plots
using RigidbodyTreeInspector

INFO: Checking out RigidBodyDynamics robocon2017...


LoadError: RigidBodyDynamics is dirty, bailing

# Creating a double pendulum `Mechanism`

We're going to create a simple `Mechanism` that represents a [double pendulum](https://en.wikipedia.org/wiki/Double_pendulum). The `Mechanism` type can be thought of as an interconnection of rigid bodies and joints.

One way of creating a `Mechanism` is by parsing a [URDF](http://wiki.ros.org/urdf) file:

In [2]:
filename = "doublependulum.urdf"
doublependulum = parse_urdf(Float64, filename)
remove_fixed_joints!(doublependulum) # remove 0-dof joints

Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow

Here are the bodies of our double pendulum:

In [9]:
collect(bodies(doublependulum))

3-element Array{RigidBodyDynamics.RigidBody{Float64},1}:
 RigidBody: "world"     
 RigidBody: "upper_link"
 RigidBody: "lower_link"

And here are its joints:

In [15]:
collect(joints(doublependulum))

2-element Array{RigidBodyDynamics.Joint{Float64},1}:
 Joint "shoulder": Revolute joint with axis [0.0,1.0,0.0]
 Joint "elbow": Revolute joint with axis [0.0,1.0,0.0]   

# The state of a `Mechanism`

A `Mechanism` stores the joint/rigid body layout, but no state information. State information is separated out into a `MechanismState` object:

In [11]:
state = MechanismState(Float64, doublependulum)

MechanismState{Float64, Float64, Float64}(…)

Let's first set the configurations and velocities of the joints:

In [16]:
shoulder, elbow = (joints(doublependulum)...)
configuration(state, shoulder)[:] = 0.3
configuration(state, elbow)[:] = 0.4
velocity(state, shoulder)[:] = 1.
velocity(state, elbow)[:] = 2.;

**Important**: a `MechanismState` contains cache variables that depend on the configurations and velocities of the joints. These need to be invalidated when the configurations and velocities are changed. To do this, call

In [17]:
setdirty!(state)

The joint configurations and velocities are stored as `Vector`s (denoted $q$ and $v$ respectively in this package) inside the `MechanismState` object:

In [18]:
@show q = configuration_vector(state)
@show v = velocity_vector(state);

q = configuration_vector(state) = [0.3,0.4]
v = velocity_vector(state) = [1.0,2.0]


# Kinematics

We are now ready to do kinematics. Here's how you transform a point at the origin of the frame after the elbow joint to world frame:

In [20]:
world = root_body(doublependulum)
transform(state, Point3D(elbow.frameAfter, zeros(SVector{3})), default_frame(world))

Point3D in "world": [-0.29552,0.25,-0.955336]

Other objects like `Wrench`es, `Twist`s, and `SpatialInertia`s, etc. can be transformed in similar fashion.

You can also ask for the homogeneous transform to world:

In [21]:
transform_to_root(state, elbow.frameAfter)

Transform3D from "after_elbow" to "world":
rotation: 0.7000000000000001 rad about [0.0,1.0,0.0], translation: [-0.29552,0.25,-0.955336]


Or a relative transform:

In [22]:
relative_transform(state, elbow.frameAfter, shoulder.frameAfter)

Transform3D from "after_elbow" to "after_shoulder":
rotation: 0.40000000000000024 rad about [2.65182e-8,1.0,0.0], translation: [0.0,0.1,-1.0]


Here's the center of mass of the double pendulum:

In [23]:
center_of_mass(state)

Point3D in "world": [-0.543749,0.2,-1.09892]

# Dynamics

A `MechanismState` can also be used to compute quantities related to the dynamics of the `Mechanism`. Here we compute the mass matrix:

In [26]:
mass_matrix(state)

2×2 Symmetric{Float64,Array{Float64,2}}:
 4.50512  2.25106
 2.25106  1.33   

Here's the documentation for `mass_matrix` by the way:

In [27]:
?mass_matrix

search: mass_matrix mass_matrix!



```julia
mass_matrix(state)

```

Compute the joint-space mass matrix (also known as the inertia matrix) of the `Mechanism` in the given state, i.e., the matrix $M(q)$ in the unconstrained joint-space equations of motion

$$
M(q) \dot{v} + c(q, v, w_\text{ext}) = \tau
$$

This method implements the composite rigid body algorithm.


Note that there is also a zero-allocation version, `mass_matrix!` (the `!` at the end of a method is a Julia convention signifying that the function is 'in-place', i.e. modifies its input data).

We can do inverse dynamics as follows (note again that there is a non-allocating version of this method as well):

In [29]:
v̇ = [2.; 3.] # the joint acceleration vector, i.e., the time derivative of the joint velocity vector v
inverse_dynamics(state, v̇)

2-element Array{Float64,1}:
 23.3164
 15.2013

# Simulation

Let's simulate the double pendulum for 5 seconds, starting from the state we defined earlier. For this, we can use the basic `simulate` function:

In [46]:
ts, qs, vs = simulate(state, 5.);

`simulate` returns a vector of times (`ts`) and associated joint configurations (`qs`) and velocities (`vs`).

*Note:* A lower level interface for simulation/ODE integration with more options is also available. Consult the documentation for more information.

We can plot the trajectories using e.g. Plots.jl:

In [47]:
plot(ts, collect(q[1] for q in qs), xlabel = "Time [s]", ylabel = "Angle [rad]", lab = "Shoulder")

We can also show a 3D visualization of the double pendulum using DrakeVisualizer: